# Download all twitter politicians

The following webpage has all the danish politician twitterprofiles  https://filip.journet.sdu.dk/twitter/politikere/

In [31]:
import requests
from bs4 import BeautifulSoup
import re #https://developers.google.com/edu/python/regular-expressions
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

In [2]:
from tqdm.notebook import tqdm
URL = 'https://filip.journet.sdu.dk/twitter/politikere/'
page = requests.get(URL)

In [20]:
soup = BeautifulSoup(page.content, 'html.parser')


## Selecting the twitternames

See that the twitter names is in a h3 style:

In [5]:
twitter_names = soup.find_all('h3')

In [6]:
#twitter_names

In [7]:
twitter_name_list=[]

In [8]:
for names in twitter_names:
   # print(names, end='\n'*2)
    match=re.search("<h3>#",str(names))
    if match:
        result_name=re.findall(".com/\w*",str(names))[0].replace(".com/","")
        twitter_name_list.append(result_name.rstrip())
        print(result_name)

vestager
larsloekke
oestergaard
PSkipperEL
IdaAuken
Kristian_Jensen
uffeelbaek
SorenPape
PiaOlsen
Heunicke
sofiecn
Statsmin
regeringDK
brianholm1962
ammitzbollbille
DanJoergensen
JakobEllemann
DanishMFA
mattiastesfaye
RosenkrantzT
EllenTraneNorby
MrMesserschmidt
FrankJensenKBH
aahlers
sophieloehde
martinlidegaard
karmel80
zeniastampe
Astridkrag
BennyEngelbrech
PHummelgaard
RasmusJarlov
JanEJoergensen
StemLAURITZEN
MogensJensenS
metteabildgaard
mfMorten
Kristianthdahl
PerClausen3
RasmusPrehn
jacobmark_sf
_MaiMercado
Jesper_Pet
nvillumsen
Trinebramsen
skaarup_df
AlexVanopslagh
SosseSass
MargreteAuken
MetteGjerskov
rohde_jens
LotteRod
PernilleVermund
mortenhelveg
UFM_MIN
espersendf
olebirkolesen
Pia_Kjaersgaard
RosaLundEl
LisbethBNielsen
rasmusnordqvist
SophieHAndersen
JeppeKofod
AneHalsboe
SchaldemoseMEP
Loekkegaard_MEP
Finansmin
engelschmidt
simonkollerup
KHegaard
khoenge
Jens_Joel
Erhvervs_min
Ulla_Tornaes
Torstenschack
DKsundhed
rasmushorn
michaelaastrup
naserkhaderdk
KofodPeter
MFVMi

## The real names

The real names is in a < small > class.

In [9]:
real_name_list=[]

In [10]:
real_names = soup.find_all('small')

In [11]:
for r_names in real_names:
    result=re.search(r'<small>.+',str(r_names)).group(0).replace("<small>","")
    real_name_list.append(result.rstrip())
    print(result)

Margrethe Vestager 
Lars Løkke Rasmussen 
Morten Østergaard 
Pernille Skipper 
Ida Auken 
Kristian Jensen 
Uffe Elbaek 
Søren Pape Poulsen 
Pia Olsen Dyhr 
Magnus Heunicke 
Sofie CarstenNielsen 
Statsministeriet 
Regeringen 
Brian Holm 
Simon Emil Ammitzbøll-Bille 
Dan Jørgensen 
Jakob Ellemann-Jensen 
Denmark MFA 🇩🇰 
Mattias Tesfaye 
Rosenkrantz-Theil 
Ellen Trane Nørby 
Morten Messerschmidt 
Frank Jensen 
Tommy Ahlers 
Sophie Løhde 
Martin Lidegaard 
Karen Melchior😷🧼🇪🇺 
Zenia Stampe 
Astrid Krag 
Benny Engelbrecht 
Peter Hummelgaard 
Rasmus Jarlov 
Jan E. Jørgensen 
Karsten Lauritzen 
Mogens Jensen 
Mette Abildgaard 
Morten Bødskov 
Kristian Thulesen Dahl 
Per Clausen 
Rasmus Prehn 
Jacob Mark 
Mai Mercado 
Jesper Petersen 
Nikolaj Villumsen 
Trine Bramsen 
Peter Skaarup 
Alex Vanopslagh 
Henrik Sass 
Margrete Auken 
Mette Gjerskov 
Jens Rohde 
Lotte Rod 
Pernille Vermund 
Morten Helveg 
Uddannelses- og Forskningsministeriet 
Søren Espersen 
Ole Birk Olesen 
Pia Kjærsgaard 
Rosa Lund

In [12]:
len(real_name_list)

651

## Making dataframe

In [13]:
d={'Names':real_name_list,"Twittername":twitter_name_list}
data=pd.DataFrame(data=d)

In [14]:
data.to_csv("danish_politkere_twitter.csv")

In [15]:
data.head()

,Names,Twittername
0,Margrethe Vestager,vestager
1,Lars Løkke Rasmussen,larsloekke
2,Morten Østergaard,oestergaard
3,Pernille Skipper,PSkipperEL
4,Ida Auken,IdaAuken


## Finding policial parties from https://www.danskepolitikere.dk/

In [78]:
# load csv
data = pd.read_csv("danish_politkere_twitter.csv")
data = data.drop(columns=['Unnamed: 0'])
soup = BeautifulSoup(requests.get('https://www.danskepolitikere.dk/oversigt/danske-politikere').content, 'html.parser')

In [79]:
sections = list(soup.find_all('section'))

In [80]:
# add "UNKNOWN" as default party
data['Party'] = 'UNKNOWN'
data.head()

,Names,Twittername,Party
0,Margrethe Vestager,vestager,UNKNOWN
1,Lars Løkke Rasmussen,larsloekke,UNKNOWN
2,Morten Østergaard,oestergaard,UNKNOWN
3,Pernille Skipper,PSkipperEL,UNKNOWN
4,Ida Auken,IdaAuken,UNKNOWN


In [81]:
parties = []
# cross reference for our politicians and check if he's in the party
for row, coloumns in data.iterrows(): 
    name = coloumns[0] 
    for section in sections:
        party = section.h3.a.text # gets the party 
        if re.search(name.lower(), str(section).lower()):
            data.at[row, 'Party'] = party
            print("Found " + name + " in " + party)
            
        


Found Margrethe Vestager in Radikale Venstre
Found Lars Løkke Rasmussen in Venstre
Found Morten Østergaard in Radikale Venstre
Found Pernille Skipper in Enhedslisten
Found Ida Auken in Radikale Venstre
Found Kristian Jensen in Venstre
Found Søren Pape Poulsen in Det Konservative Folkeparti
Found Pia Olsen Dyhr in Socialistisk Folkeparti
Found Magnus Heunicke in Socialdemokratiet
Found Statsministeriet in Radikale Venstre
Found Statsministeriet in Socialdemokratiet
Found Statsministeriet in Venstre
Found Dan Jørgensen in Socialdemokratiet
Found Jakob Ellemann-Jensen in Venstre
Found Mattias Tesfaye in Socialdemokratiet
Found Rosenkrantz-Theil in Socialdemokratiet
Found Ellen Trane Nørby in Venstre
Found Morten Messerschmidt in Dansk Folkeparti
Found Frank Jensen in Socialdemokratiet
Found Tommy Ahlers in Venstre
Found Sophie Løhde in Venstre
Found Martin Lidegaard in Radikale Venstre
Found Zenia Stampe in Radikale Venstre
Found Astrid Krag in Socialdemokratiet
Found Benny Engelbrecht in

In [82]:
# view results
data.groupby('Party').count()

,Names,Twittername
Party,,
Alternativet,1,1
Dansk Folkeparti,13,13
Det Konservative Folkeparti,14,14
Enhedslisten,12,12
Inuit Ataqatigiit,1,1
Liberal Alliance,5,5
Radikale Venstre,15,15
Socialdemokratiet,42,42
Socialistisk Folkeparti,19,19


In [83]:
# get snapshot of who's missing
data.loc[data['Party'] == "UNKNOWN"]



,Names,Twittername,Party
6,Uffe Elbaek,uffeelbaek,UNKNOWN
10,Sofie CarstenNielsen,sofiecn,UNKNOWN
12,Regeringen,regeringDK,UNKNOWN
13,Brian Holm,brianholm1962,UNKNOWN
14,Simon Emil Ammitzbøll-Bille,ammitzbollbille,UNKNOWN
...,...,...,...
646,Molle lykke nielsen,LykkeMolle,UNKNOWN
647,Vibeke Gamst,vibeke_gamst,UNKNOWN
648,Pia Moldt,piamoldt,UNKNOWN
649,Lars Folmann,larsfolm,UNKNOWN


Too many still unkown so we must look elsewhere, in denmark we have many
regions with politicans that aren't necsarily members of the parlament
![Regioner i dk](https://rn.dk/-/media/Rn_dk/Genveje/Fakta-om-Nordjylland/Regioner-i-Danmark/Danmark_Regioner_2017.ashx?la=da)
So we will scrape these websites as well


### first north jutland

In [84]:
nordjylland_url = "https://rn.dk/da/Politik/De-regionale-politikere"

soup = BeautifulSoup(requests.get(nordjylland_url).content, 'html.parser')



In [85]:
# examine html
#print(soup.prettify())
match_rnj = []
pre_re = r"\b[A-Z].*"
div_lsit = soup.findAll("ul", {'class': 'span3'})
for paragraph in div_lsit:
    text= paragraph.text
    text_stripped = text.rstrip().lstrip()
    #print(text_stripped)
    matches_1 = re.findall(pre_re, text_stripped)
    print(matches_1)
    match_rnj.append(matches_1)

# 0 and even indices are names, 1 and odd are the matching parti strings

['Budget og regnskab', 'Dine rettigheder når regionen behandler oplysninger om dig', 'Digital post', 'Indkøb', 'Licitationer og udbud']
['KlimaRegion', 'Organisation', 'Strategier og politikker', 'Samarbejde med erhvervsliv']
['Design', 'Sociale medier', 'Vision og retning', 'Sundhedstegn - podcast']
['De regionale politikere', 'Regionsrådet', 'Se Regionsrådsmøder', 'Dialog med politikerne i Region Nordjylland']
['Politiske udvalg under Regionsrådet', 'Forretningsudvalget', 'Kontaktudvalget', 'Sundhedskoordinationsudvalget']
['Vederlag for politisk arbejde', 'Partistøtte']
['Akut sygdom', 'Hospitaler']
['Dine rettigheder som patient', 'Praktiserende læger og behandlere']
['NemSMS']
['Forskning', 'Akuthjælperordningen', 'Kontaktliste for kommunernes hjemmepleje', 'Kontaktliste til kommuner på børneområdet', 'National og Tværsektoriel IT', 'Nord-KAP']
['Samskabelse', 'Planer på sundhedsområdet', 'Samarbejde med kommuner omkring infektionshygiejne', 'Folkesundhed', 'Sundhedsaftalen 2019',

In [94]:
# note that relevant matches come at last 2 indices
good_match = match_rnj[len(match_rnj)-2] + match_rnj[len(match_rnj)-1]

print(good_match)
# remove strings that contain "formand" as it's not relevant information
good_match = [x for x in good_match if "formand" not in x.lower() if "\r" not in x.lower()]
print(good_match)
print(len(good_match))

for i in range(len(good_match)-1):
    name = good_match[i]
    party_string = good_match[i+1]
    if name in data.values:
        print(name+ "found in "+ party_string.replace("Parti: ", ""))
        data.loc[data['Names'] == name, 'Party'] = party_string.replace("Parti: ", "")



['Ulla Astman', 'Parti: Socialdemokratiet', 'Regionsrådsformand', 'Lone Sondrup', 'Parti: Venstre, Danmarks Liberale Parti', 'Næstformand', 'Lene Linnemann', 'Parti: SF - Socialistisk Folkeparti     ', 'Næstformand', 'Anne Mette Mortensen', 'Parti: Dansk Folkeparti', 'Anny Winther', 'Parti: Venstre, Danmarks Liberale Parti', 'Arne Nielsen', 'Parti: Socialdemokratiet', 'Bente Bang', 'Parti: Socialdemokratiet', 'Ejner Guldager Nielsen', 'Parti: Socialdemokratiet', 'Erik Harbo Larsen', 'Parti: Venstre, Danmarks Liberale Parti', 'Erik Høgh-Sørensen\r', 'Erik Høgh-Sørensen', 'Parti: Dansk Folkeparti', 'Gitte Lopdrup', 'Parti: Radikale Venstre', 'Hanne Korsgaard', 'Parti: Socialdemokratiet', 'Henrik Buchhave', 'Parti: Venstre, Danmarks Liberale Parti', 'Ib Madsen', 'Parti: Dansk Folkeparti', 'Jess V. Laursen', 'Parti: Venstre, Danmarks Liberale Parti', 'Jette Ramskov', 'Parti: Socialdemokratiet', 'Jørgen Hammer Sørensen', 'Parti: Dansk Folkeparti', 'Jørgen Hansen', 'Parti: Socialdemokratiet'

In [96]:
data.groupby('Party').count()

,Names,Twittername
Party,,
10,1,1
Alternativet,1,1
Dansk Folkeparti,14,14
Det Konservative Folkeparti,16,16
Enhedslisten,12,12
Inuit Ataqatigiit,1,1
Liberal Alliance,5,5
Radikale Venstre,15,15
SF - Socialistisk Folkeparti,1,1


# Region Midtjylland

In [148]:
url = 'https://www.rm.dk/politik/regionsradsmedlemmer/'

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

content = soup.find("div", {"class": "text-content"})

unwanted = content.findAll("p")
for u in unwanted:
    u.clear()

# removing some introductioniary text
unwanted = content.find("h2")
print(unwanted.clear())
print(unwanted.find_next().clear())
clean_content = content.text.rstrip()

None
None


In [154]:
# removing unwanted brackets ()
pattern = r"\(.*?\)"
clean_content = re.sub(pattern, '', clean_content)


print(clean_content) # left with this string

<h3>Socialdemokratiet (A)</h3>





Socialdemokratiet 


Anders Kühnau
Annette Roed
Birgit Marie Christensen
Birgitte Svenningsen 
Claus Kjeldsen
Conny Jensen
Flemming Knudsen
Henrik Fjeldgaard
Henrik Gottlieb Hansen
John G. Christensen
Lars Møller Pedersen
Marianne Carøe
Mette Valbjørn
Morten Flæng
Niels Erik Iversen
Ole Jepsen 

Radikale Venstre 


Hanne Roed

Det Konservative Folkeparti 


Nicolaj Bang

Socialistisk Folkeparti 


Jacob Isøe Klærke
Susanne Buch

Liberal Alliance 


Jakob Rixen

Kristendemokraterne 


Marianne Karlsmose

Dansk Folkeparti 


Lone Langballe
Ole Revsgaard Andersen
Steen Thomsen

Psykiatri-Listen 


Mikkel Rasmussen 
Peter Møller Andersen 

Venstre 


Arne Lægaard
Carsten Kissmeyer
Christian Møller-Nielsen
Dorte West
Erik Vinther
Finn Thranum
Ib Bjerregaard 
Jørgen Nørby
Jørgen Winther
Olav Nørgaard
Steen Jakobsen
Torben Nørregaard
Ulrich Fredberg

Enhedslisten 


Else Kayser
Henrik Qvist 
Signe Lund Jensen 

Alternativet 


Rasmus Foged


In [188]:
all_parties = content.findAll("h3")
for party in all_parties:
    #print(party.text)
    party_text = re.sub(pattern,'', party.text)
    people = party.find_next('ul')
    people_text = re.sub(pattern, "", people.text)
    for name in people_text.split('\n'):
        if name in data.values:
            print(name + "  in   " + party_text)
            data.loc[data['Names'] == name, 'Party'] = party_text.rstrip()

Anders Kühnau  in   Socialdemokratiet 
Annette Roed  in   Socialdemokratiet 
Claus Kjeldsen  in   Socialdemokratiet 
Mette Valbjørn  in   Socialdemokratiet 
Hanne Roed  in   Radikale Venstre 
Nicolaj Bang  in   Det Konservative Folkeparti 
Jacob Isøe Klærke  in   Socialistisk Folkeparti 
Jakob Rixen  in   Liberal Alliance 
Lone Langballe  in   Dansk Folkeparti 
Ole Revsgaard Andersen  in   Dansk Folkeparti 
Arne Lægaard  in   Venstre 
Carsten Kissmeyer  in   Venstre 
Dorte West  in   Venstre 
Steen Jakobsen  in   Venstre 
Ulrich Fredberg  in   Venstre 
Else Kayser  in   Enhedslisten 
Rasmus Foged  in   Alternativet 


In [189]:
data.groupby('Party').count()

,Names,Twittername
Party,,
10,1,1
Alternativet,2,2
Dansk Folkeparti,16,16
Det Konservative Folkeparti,17,17
Enhedslisten,13,13
Inuit Ataqatigiit,1,1
Liberal Alliance,6,6
Radikale Venstre,16,16
SF - Socialistisk Folkeparti,1,1


In [187]:
data.Party.unique()

array(['Radikale Venstre', 'Venstre', 'Enhedslisten', 'UNKNOWN',
       'Det Konservative Folkeparti', 'Socialistisk Folkeparti',
       'Socialdemokratiet', 'Dansk Folkeparti', 'Liberal Alliance',
       'Alternativet', 'Socialdemokratiet ', 'Inuit Ataqatigiit',
       'Socialistisk Folkeparti ', 'Radikale Venstre ', 'Venstre ',
       'Alternativet ', 'Uden for folketingsgrupperne',
       'Venstre, Danmarks Liberale Parti', 'Liberal Alliance ',
       'Dansk Folkeparti ', 'Det Konservative Folkeparti ',
       'Enhedslisten ', 'SF - Socialistisk Folkeparti     ', '10', 'Test'],
      dtype=object)